# DSCI 525 - Web and Cloud Computing 

### Milestone 1: Tackling big data on your laptop

### Group 13: Kelly Wu, Julie Song, Bruce Wu, Vincent Ho

### 3. Downloading the data 
rubric={correctness:10}

1. Download the data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) to your local computer using the [figshare API](https://docs.figshare.com) (you need to make use of `requests` library).

2. Extract the zip file, again programmatically, similar to how we did it in class. 

>  You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the `requests` library, similar to how we [did it in class](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture).

> There are 5 files in the figshare repo. The one we want is: `data.zip`

In [35]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np

In [2]:
article_id = 14096681

In [3]:
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.33 s, sys: 6.83 s, total: 11.2 s
Wall time: 16min 33s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.7 s, sys: 1.1 s, total: 8.8 s
Wall time: 8.86 s


### 4. Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.
    
2. When combining the CSV files, add an extra column called "model" that identifies the model.
    Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON
    Tip 2: Remember how we added "year" column when we combined airline CSVs. Here the regex will be to get word before an underscore ie, "/([^_]*)"

> Note: There is a file called `observed_daily_rainfall_SYD.csv` in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from the folder) before you combine CSVs. We will use this file in our next milestone.

3. ***Compare*** run times on different machines within your team and summarize your observations. 

> Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop. 

In [7]:
# Remove observed_daily_rainfall_SYD.csv in the downloaded folder

os.remove('figsharerainfall/observed_daily_rainfall_SYD.csv')

In [8]:
%%time

files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )

df.to_csv("figsharerainfall/combined_data.csv")

CPU times: user 3min 20s, sys: 10.5 s, total: 3min 31s
Wall time: 3min 33s


In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.6G	figsharerainfall/combined_data.csv


In [10]:
print(df.shape)

(62467843, 6)


In [11]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [17]:
time_diff_four = {'Team Member': ['Julie', 'Kelly', 'Bruce', 'Vincent'],
        'Operating System': ['MacOS', 'MacOS', 'Windows11', 'MacOS'],
        'RAM': ['16GB', '16GB', '16GB', '8GB'],
        'Processor': ['Apple M1 Pro', '2.8 GHz Quad-Core Intel Core i7', '12th Gen Intel(R) Core(TM) i7-12700H', 'Apple M1'],
        'Is SSD': ['Yes', 'Yes', 'Yes', 'Yes'],
        'Time taken': ['4min 33s', '4min 50s', '12min 4s', '3min 33s']}

pd.DataFrame(time_diff_four)

,Team Member,Operating System,RAM,Processor,Is SSD,Time taken
0,Julie,MacOS,16GB,Apple M1 Pro,Yes,4min 33s
1,Kelly,MacOS,16GB,2.8 GHz Quad-Core Intel Core i7,Yes,4min 50s
2,Bruce,Windows11,16GB,12th Gen Intel(R) Core(TM) i7-12700H,Yes,12min 4s
3,Vincent,MacOS,8GB,Apple M1,Yes,3min 33s


Among the processors, it appears that the task was generally performed more efficiently by MacOS's processors, as they took half the time to combine different CSV files into a single CSV with a size of 5.6GB compared to Windows' processors. While it seems that the Apple M1 chip performed more efficiently than the Intel chip among MacOS processors, the difference is not significant when comparing to MacOS versus Windows11.

In [13]:
%reset -f
## This is to clear the memory, and you get a fresh start. When you run this, you will lose all the variables that you have created so far.

### 5. Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts). 
    - Changing `dtype` of your data
    - Load just columns that we want
    - Loading in chunks
    
2. ***Compare*** run times on different machines within your team and summarize your observations.

In [14]:
import pandas as pd
import numpy as np

In [15]:
# Load three columns only: "time", "rain (mm/day)", "model"
# Changing dtype of 'rain (mm/day)' from float64 to float32

%%time
use_cols = ["time", "rain (mm/day)", 'model']
df_only3col = pd.read_csv("figsharerainfall/combined_data.csv",usecols=use_cols, 
                 dtype = {'rain (mm/day)': np.float32})
print(df_only3col["model"].value_counts())
print(df_only3col.info())
print(df_only3col.describe())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   rain (mm/day)  

In [18]:
time_diff_five = {'Team Member': ['Julie', 'Kelly', 'Bruce', 'Vincent'],
        'Operating System': ['MacOS', 'MacOS', 'Windows11', 'MacOS'],
        'RAM': ['16GB', '16GB', '16GB', '8GB'],
        'Processor': ['Apple M1 Pro', '2.8 GHz Quad-Core Intel Core i7', '12th Gen Intel(R) Core(TM) i7-12700H', 'Apple M1'],
        'Is SSD': ['Yes', 'Yes', 'Yes', 'Yes'],
        'Time taken': ['47.2 s', '59.6 s', '1min 16s', '35.3 s']}

pd.DataFrame(time_diff_five)

,Team Member,Operating System,RAM,Processor,Is SSD,Time taken
0,Julie,MacOS,16GB,Apple M1 Pro,Yes,47.2 s
1,Kelly,MacOS,16GB,2.8 GHz Quad-Core Intel Core i7,Yes,59.6 s
2,Bruce,Windows11,16GB,12th Gen Intel(R) Core(TM) i7-12700H,Yes,1min 16s
3,Vincent,MacOS,8GB,Apple M1,Yes,35.3 s


Among the processors, it appears that the task was generally performed more efficiently by MacOS's processors, as they took less time to load the combined CSV with 5.6GB into memory and perform a simple EDA. While it seems that MacOS's processors performed more efficiently than Windows' processors, the difference is not significant when comparing the results of combining the CSV files.

### 6. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}

1. Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others.
    - [Parquet file](http://parquet.apache.org)
    - [Pandas exchange](https://rpy2.github.io/doc/latest/html/interactive.html)
    - [Arrow exchange](https://github.com/rpy2/rpy2-arrow)
2. Once you have the dataframe in R, perform a simple EDA.

### Reasoning:

We will choose Arrow exchange since Arrow is optimized for performance and can take advantage of multi-core processors and distributed systems to speed up data transfer. One of the key benefits of Arrow is that it supports zero-copy data sharing between different programming languages. This means that the data can be transferred between Python and R without the need for copying or converting the data, which can be time-consuming and memory-intensive. Besides, Arrow has good support for both Python and R, with libraries available for both languages, which means that it's easy to integrate Arrow into existing Python and R workflows.
One disadvantage of using Pandas exchange is that it can be memory-intensive, especially for large datasets. When transferring data between Python and R using Pandas exchange, the entire dataset must be loaded into memory on both sides, which is not available for our dataset with 6GB. Moreover, Pandas exchange is not as efficient as Arrow exchange since it relies on serializing and deserializing data, which can be slower and less efficient than zero-copy data sharing.
The disadvantage of Parquet file in our dataset could be the low efficiency of using Parquet file for our dataset as it spends much more time than the Arrow exchange. Moreover, the Arrow exchange uses a flat memory layout, which means that the memory required to store the data is minimized, whereas Parquet file has a more complex memory layout that requires more memory to store the same data.

In [3]:
%reset -f

In [4]:
filepathcsv = "/Users/vincentho/Desktop/DSCI525_group13/notebooks/figsharerainfall/combined_data.csv"

In [5]:
import os
os.environ['R_HOME'] = '/Users/vincentho/opt/miniconda3/envs/525_2023/lib/R'

In [6]:
%load_ext rpy2.ipython

In [7]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [8]:
%%time
dataset = ds.dataset(filepathcsv, format="csv")

table = dataset.to_table()

r_table = pyra.converter.py2rpy(table)

CPU times: user 12.2 s, sys: 1.3 s, total: 13.5 s
Wall time: 13 s


In [14]:
%%time
%%R -i r_table
start_time <- Sys.time()
suppressMessages(library(dplyr))


result <- r_table 

end_time <- Sys.time()

print(result %>% collect())
print(end_time - start_time)

# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1889-01-01 04:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-…
 2 1889-01-02 04:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-…
 3 1889-01-03 04:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-…
 4 1889-01-04 04:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-…
 5 1889-01-05 04:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-…
 6 1889-01-06 04:00:00   -35.4   -33.6    142.    143.        4.20e-13 MPI-ESM-…
 7 1889-01-07 04:00:00   -35.4   -33.6    142.    143.        4.19e-13 MPI-ESM-…
 8 1889-01-08 04:00:00   -35.4   -33.6    142.    143.        4.56e-13 MPI-ESM-…
 9 1889-01-09 04:00:00   -35.4   -33.6    142.    143.        2.53e+ 0 MPI-ESM-…
10 1889-01-10 04:00:00   -35.4   -33.6    142.    143.        4.12e- 2 MPI-ESM-…
#

In [47]:
%%time
%%R -i r_table
start_time <- Sys.time()
suppressMessages(library(dplyr))


result <- r_table %>% group_by(model) %>% summarize(count=n(), average_rainfall = mean(`rain (mm/day)`, na.rm = TRUE)) %>% arrange(desc(average_rainfall))

end_time <- Sys.time()

print(result %>% collect())
print(end_time - start_time)

# A tibble: 27 × 3
   model           count average_rainfall
   <chr>           <int>            <dbl>
 1 INM-CM4-8     1609650             2.81
 2 INM-CM5-0     1609650             2.67
 3 CMCC-CM2-SR5  3541230             2.38
 4 MIROC6        2070900             2.30
 5 CMCC-CM2-HR4  3541230             2.28
 6 CMCC-ESM2     3541230             2.27
 7 NorESM2-MM    3541230             2.23
 8 NorESM2-LM     919800             2.23
 9 TaiESM1       3541230             2.22
10 ACCESS-ESM1-5 1610700             2.22
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
Time difference of 0.04090595 secs
CPU times: user 1.6 s, sys: 1.06 s, total: 2.66 s
Wall time: 732 ms


## Challenges or Difficulties

One of the challenges of dealing with big data is that we have to always pay attention to the amount of storage space required to store the data. It is important as repeating steps or code chunks such as combining a new 5.6GB CSV file could quickly fill up storage space and cause the laptop to crash, especially if it has limited storage capacity.
Another challenge of working with big data is the time-consuming nature of the process. It can take a considerable amount of time to realize that the code at the bottom of a program is not working properly, and then we will need to spend a significant amount of additional time rerunning the entire process again.